# BT AVRCP Metadata Analysis

Bluetooth AVRCP で取得したメタデータの分析ノートブック。
`data/` ディレクトリ内の JSONL ファイルを読み込み、サービス・端末・OS ごとのメタデータ充実度を可視化する。

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

sns.set_theme(style="darkgrid")
plt.rcParams["font.family"] = ["Hiragino Sans", "IPAGothic", "sans-serif"]

DATA_DIR = Path("../data")
print(f"データディレクトリ: {DATA_DIR.resolve()}")
print(f"JSONL ファイル数: {len(list(DATA_DIR.glob('*.jsonl')))}")

## 1. データ読み込み

全 JSONL ファイルからセッションヘッダーとトラックデータを読み込み、pandas DataFrame に変換する。

In [ ]:
sessions = []
tracks = []

for filepath in sorted(DATA_DIR.glob("*.jsonl")):
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            record = json.loads(line)
            if record.get("type") == "session_header":
                record["filename"] = filepath.name
                sessions.append(record)
            elif record.get("type") == "track":
                record["filename"] = filepath.name
                tracks.append(record)

df_sessions = pd.DataFrame(sessions)
df_tracks = pd.DataFrame(tracks)

print(f"セッション数: {len(df_sessions)}")
print(f"トラック数: {len(df_tracks)}")

if not df_sessions.empty:
    display(df_sessions.head())

## 2. セッション情報をトラックに結合

各トラックにセッションのメタ情報（content_name, device, os_version 等）を付与する。

In [ ]:
if not df_sessions.empty and not df_tracks.empty:
    session_meta = df_sessions[["filename", "content_name", "platform_type", "device", "os_version"]]
    df = df_tracks.merge(session_meta, on="filename", how="left")
    print(f"結合後のトラック数: {len(df)}")
    display(df.head())
else:
    df = pd.DataFrame()
    print("データがありません。セッションを記録してください。")

## 3. メタデータ充実度ヒートマップ

サービス × メタデータフィールドの取得率をヒートマップで可視化する。

In [ ]:
METADATA_FIELDS = ["title", "artist", "album", "genre", "track_number", "number_of_tracks", "duration_ms"]


def has_value(val):
    """有意な値があるか判定する。"""
    if pd.isna(val):
        return False
    if isinstance(val, str) and not val.strip():
        return False
    if isinstance(val, (int, float)) and val == 0:
        return False
    return True


if not df.empty:
    coverage_data = []
    for service, group in df.groupby("content_name"):
        row = {"service": service}
        for field in METADATA_FIELDS:
            if field in group.columns:
                rate = group[field].apply(has_value).mean() * 100
            else:
                rate = 0.0
            row[field] = round(rate, 1)
        coverage_data.append(row)

    df_coverage = pd.DataFrame(coverage_data).set_index("service")

    fig, ax = plt.subplots(figsize=(12, max(4, len(df_coverage) * 0.8)))
    sns.heatmap(
        df_coverage,
        annot=True,
        fmt=".0f",
        cmap="RdYlGn",
        vmin=0,
        vmax=100,
        linewidths=0.5,
        ax=ax,
    )
    ax.set_title("メタデータ充実度 (%) - サービス別")
    ax.set_ylabel("")
    plt.tight_layout()
    plt.show()
else:
    print("データがありません。")

## 4. サービス × 端末 × OS クロス集計

同じサービスでも端末・OS によるメタデータの違いを確認する。

In [ ]:
if not df.empty:
    cross_data = []
    for (content, device, os_ver), group in df.groupby(["content_name", "device", "os_version"]):
        row = {
            "content": content,
            "device": device,
            "os": os_ver,
            "tracks": len(group),
        }
        for field in METADATA_FIELDS:
            if field in group.columns:
                rate = group[field].apply(has_value).mean() * 100
            else:
                rate = 0.0
            row[field] = round(rate, 1)
        cross_data.append(row)

    df_cross = pd.DataFrame(cross_data)
    display(df_cross)
else:
    print("データがありません。")

## 5. Duration 分布

サービスごとの duration_ms の分布を箱ひげ図で確認する。

In [ ]:
if not df.empty and "duration_ms" in df.columns:
    df_dur = df[df["duration_ms"].apply(has_value)].copy()
    df_dur["duration_sec"] = df_dur["duration_ms"] / 1000

    if not df_dur.empty:
        fig, ax = plt.subplots(figsize=(12, 6))
        sns.boxplot(data=df_dur, x="content_name", y="duration_sec", ax=ax)
        ax.set_title("Duration 分布 (秒) - サービス別")
        ax.set_xlabel("サービス")
        ax.set_ylabel("Duration (秒)")
        plt.xticks(rotation=45, ha="right")
        plt.tight_layout()
        plt.show()
    else:
        print("有効な duration データがありません。")
else:
    print("データがありません。")

## 6. まとめ

上記の分析から、各サービス・端末・OS でどのメタデータフィールドが利用可能かを把握できる。  
特に注目すべきポイント:

- **Album / Genre**: アプリとプラットフォームによって送信の有無が大きく異なる
- **TrackNumber / NumberOfTracks**: ストリーミングサービスでは送信されないケースが多い
- **Duration**: YouTube 等の動画系サービスでは 0 が返されることがある